In [11]:
import pandas as pd
import numpy as np
import ants

from aidream_registration import constants
from aidream_registration.dataloaders import AtlasImagingNiftiLoader
from aidream_registration.utils.cohort_utils import get_perfusion_patients

In [12]:
# First, load the hammersmith atlas, and create a mask :
path_hammersmith = constants.DIR_DATA / "hammersmith" / "T1w_ICBM_skullstripped.nii.gz"
ants_hammersmith = ants.image_read(str(path_hammersmith))

mask_hammersmith = ants_hammersmith > 0
print(f"Hammersmith mask volume: {mask_hammersmith.sum() / 1e3:.2f} cm3")


Hammersmith mask volume: 1886.57 cm3


In [13]:
# Load all segmentations in a dictionary :
dict_segmentations = {
    path_seg.name.removesuffix(".nii.gz"): ants.image_read(str(path_seg)) > 0
    for path_seg in (constants.DIR_DATA / "hammersmith").glob("*.nii.gz")
    if path_seg.name != "T1w_ICBM_skullstripped.nii.gz"
}

print("Number of segmentations:", len(dict_segmentations))

for segmentation, mask_segmentation in dict_segmentations.items():
    print(fr"{segmentation} volume: {mask_segmentation.sum() / 1e3:.2f} cm3")


Number of segmentations: 21
L.Ventricles volume: 6.72 cm3
R_insula volume: 16.48 cm3
R.Ventricles volume: 6.11 cm3
L.Temporal_lobe volume: 115.72 cm3
R.Occipital_lobe volume: 76.46 cm3
L.Grey_nuclei volume: 18.22 cm3
R.Grey_nuclei volume: 18.20 cm3
R.Limbic_lobe volume: 24.28 cm3
Brainstem volume: 26.94 cm3
L.frontal_lobe volume: 225.46 cm3
third_ventricle volume: 0.63 cm3
L.Post_fossea volume: 90.63 cm3
L.Parietal_lobe volume: 142.35 cm3
Corpus_callosum volume: 21.05 cm3
R.Temporal_lobe volume: 121.76 cm3
L.Occipital_lobe volume: 75.31 cm3
R.frontal_lobe volume: 227.50 cm3
R.Post_fossea volume: 88.57 cm3
R.Parietal_lobe volume: 141.59 cm3
L.Limbic_lobe volume: 24.96 cm3
L_insula volume: 16.62 cm3


In [6]:
# Create mask_1 for the ventricle voxels :
mask_1 = dict_segmentations["L.Ventricles"] + dict_segmentations["R.Ventricles"] + dict_segmentations["third_ventricle"]
mask_1 = mask_1 > 0

print(f"Ventricles volume: {mask_1.sum() / 1e3:.2f} cm3")


Ventricles volume: 13.47 cm3


In [7]:
# Create mask82 for the segmented voxels that are not ventricles :
mask_2 = None
for seg, mask_segmentation in dict_segmentations.items():

    if seg not in ["L.Ventricles", "R.Ventricles", "third_ventricle"]:
        if mask_2 is None:
            mask_2 = mask_segmentation
        else:
            mask_2 += mask_segmentation

mask_2 = mask_2 > 0

print(f"Brain volume: {mask_2.sum() / 1e3:.2f} cm3")

mask_2.to_file(str(constants.DIR_DATA / "hammersmith" / "custom" / "mask_2.nii.gz"))


Brain volume: 1472.11 cm3


In [ ]:
# Create two bounding boxes for the ventricles , a small one and a big one :

seg_idx = np.array(np.nonzero(mask_1.numpy()))

bbox_min, bbox_max = np.min(seg_idx, axis=1), np.max(seg_idx, axis=1)

bbox_min_1, bbox_max_1 = bbox_min - 3, bbox_max + 3
bbox_min_2, bbox_max_2 = bbox_min - 8, bbox_max + 8

np_bbox_1 = np.zeros(mask_1.shape)
np_bbox_1[bbox_min_1[0]: bbox_max_1[0], bbox_min_1[1]: bbox_max_1[1], bbox_min_1[2]: bbox_max_1[2]] = 1

np_bbox_2 = np.zeros(mask_1.shape)
np_bbox_2[bbox_min_2[0]: bbox_max_2[0], bbox_min_2[1]: bbox_max_2[1], bbox_min_2[2]: bbox_max_2[2]] = 1

mask_small = (ants.from_numpy(np_bbox_1, origin=mask_1.origin, spacing=mask_1.spacing, direction=mask_1.direction) > 0) * mask_hammersmith
mask_big = (ants.from_numpy(np_bbox_2, origin=mask_1.origin, spacing=mask_1.spacing, direction=mask_1.direction) > 0) * mask_hammersmith
